# Data Preparation 2019: Merge Synthetic Water Data with Real Health Data

This notebook merges the synthetic JJM FHTC coverage data with the real health data for the 2019-2020 baseline period.

**Objectives:**
1. Load synthetic water data (jjm_raw_2019.csv)
2. Load real health data (health_2019_cleaned.csv) and filter for "Inpatient" indicators
3. Clean and standardize data (numeric types, district name matching)
4. Merge datasets on District_Name and Month
5. Save the final merged panel dataset


In [ ]:
# Import required libraries
import pandas as pd
import numpy as np
from pathlib import Path
import sys
import difflib
import warnings
warnings.filterwarnings('ignore')

# Add parent directory to path for config imports
sys.path.insert(0, str(Path.cwd().parent if Path.cwd().name == 'notebooks' else Path.cwd()))

try:
    from config import FILE_PATHS
    print("✓ Config imported successfully")
except ImportError:
    # Fallback paths if config not available
    FILE_PATHS = {
        "data": {
            "raw": "data/raw",
            "processed": "data/processed"
        }
    }
    print("⚠ Using fallback paths")

print("✓ Libraries imported successfully")


## Step 1: Load Data


In [ ]:
# Load synthetic JJM data (Water Data)
jjm_file = Path(FILE_PATHS["data"]["raw"]) / "jjm_raw_2019.csv"
print(f"Loading synthetic JJM data from: {jjm_file}")

try:
    jjm_df = pd.read_csv(jjm_file, parse_dates=['Date'])
    print(f"✓ JJM data loaded: {jjm_df.shape[0]:,} rows, {jjm_df.shape[1]} columns")
    print(f"  Columns: {list(jjm_df.columns)}")
    print(f"\nFirst few rows:")
    display(jjm_df.head())
except FileNotFoundError:
    print(f"❌ Error: File not found at {jjm_file}")
    print("Please run src/generate_synthetic_jjm.py first to generate the data")
    jjm_df = None
except Exception as e:
    print(f"❌ Error loading JJM data: {e}")
    jjm_df = None


In [ ]:
# Load Health Data
health_file = Path(FILE_PATHS["data"]["processed"]) / "health_2019_cleaned.csv"
print(f"\nLoading health data from: {health_file}")

try:
    health_df = pd.read_csv(health_file)
    print(f"✓ Health data loaded: {health_df.shape[0]:,} rows, {health_df.shape[1]} columns")
    print(f"  Columns: {list(health_df.columns)}")
    print(f"\nFirst few rows:")
    display(health_df.head())
except FileNotFoundError:
    print(f"❌ Error: File not found at {health_file}")
    print("Please run src/process_health_2019_final.py first to generate the data")
    health_df = None
except Exception as e:
    print(f"❌ Error loading health data: {e}")
    health_df = None


# Health Data Filter: Keep ONLY rows where Indicator contains "Inpatient" (case-insensitive)
if health_df is not None:
    print(f"\nFiltering health data for 'Inpatient' indicators...")
    print(f"  Original rows: {len(health_df):,}")
    
    if 'Indicator' in health_df.columns:
        # Filter for rows where Indicator contains "Inpatient" (case-insensitive)
        health_df_filtered = health_df[
            health_df['Indicator'].astype(str).str.lower().str.contains('inpatient', na=False)
        ].copy()
        
        print(f"  Filtered rows: {len(health_df_filtered):,}")
        print(f"  Rows removed: {len(health_df) - len(health_df_filtered):,}")
        
        if len(health_df_filtered) > 0:
            print(f"\n  Sample filtered data:")
            display(health_df_filtered.head())
        else:
            print(f"  ⚠ Warning: No rows found with 'Inpatient' in Indicator column")
            health_df_filtered = None
    else:
        print(f"  ❌ Error: 'Indicator' column not found in health data")
        health_df_filtered = None
else:
    health_df_filtered = None


In [ ]:
## Step 2: Clean & Standardize


In [ ]:
# Water Data: Ensure FHTC_Coverage is numeric
if jjm_df is not None:
    print("Cleaning Water Data...")
    jjm_df_clean = jjm_df.copy()
    
    # Ensure FHTC_Coverage is numeric
    if 'FHTC_Coverage' in jjm_df_clean.columns:
        jjm_df_clean['FHTC_Coverage'] = pd.to_numeric(jjm_df_clean['FHTC_Coverage'], errors='coerce')
        print(f"  ✓ FHTC_Coverage converted to numeric")
        print(f"  Missing values: {jjm_df_clean['FHTC_Coverage'].isnull().sum()}")
    else:
        print(f"  ⚠ Warning: FHTC_Coverage column not found")
    
    # Create Month column from Date if needed
    if 'Date' in jjm_df_clean.columns and 'Month' not in jjm_df_clean.columns:
        jjm_df_clean['Month'] = jjm_df_clean['Date'].dt.strftime('%B')
        print(f"  ✓ Created Month column from Date")
    
    print(f"  Final shape: {jjm_df_clean.shape}")
else:
    jjm_df_clean = None


# Health Data: Ensure Cases is numeric
if health_df_filtered is not None:
    print("\nCleaning Health Data...")
    health_df_clean = health_df_filtered.copy()
    
    # Ensure Cases is numeric
    if 'Cases' in health_df_clean.columns:
        health_df_clean['Cases'] = pd.to_numeric(health_df_clean['Cases'], errors='coerce')
        print(f"  ✓ Cases converted to numeric")
        print(f"  Missing values: {health_df_clean['Cases'].isnull().sum()}")
    else:
        print(f"  ⚠ Warning: Cases column not found")
    
    print(f"  Final shape: {health_df_clean.shape}")
else:
    health_df_clean = None


In [ ]:
# District Names: Use difflib (fuzzy matching) to create a dictionary mapping Health District names to Water District names
if jjm_df_clean is not None and health_df_clean is not None:
    print("\nCreating district name mapping using fuzzy matching...")
    
    # Get unique district names from both datasets
    water_districts = sorted(jjm_df_clean['District_Name'].dropna().unique())
    health_districts = sorted(health_df_clean['District_Name'].dropna().unique())
    
    print(f"  Water districts: {len(water_districts)}")
    print(f"  Health districts: {len(health_districts)}")
    
    # Create mapping dictionary using fuzzy matching
    district_mapping = {}
    unmatched_health = []
    
    for health_district in health_districts:
        # Find the best match in water districts
        matches = difflib.get_close_matches(
            health_district, 
            water_districts, 
            n=1, 
            cutoff=0.6  # Minimum similarity threshold
        )
        
        if matches:
            district_mapping[health_district] = matches[0]
        else:
            unmatched_health.append(health_district)
            # If no good match, keep original name
            district_mapping[health_district] = health_district
    
    print(f"  ✓ Created mapping for {len(district_mapping)} districts")
    if unmatched_health:
        print(f"  ⚠ {len(unmatched_health)} districts could not be matched (keeping original names)")
        print(f"    Sample unmatched: {unmatched_health[:5]}")
    
    # Apply mapping to Health dataframe
    health_df_clean['District_Name'] = health_df_clean['District_Name'].map(district_mapping).fillna(health_df_clean['District_Name'])
    print(f"  ✓ Applied mapping to health data")
    
    # Show sample mappings
    print(f"\n  Sample mappings:")
    sample_mappings = list(district_mapping.items())[:10]
    for orig, mapped in sample_mappings:
        if orig != mapped:
            print(f"    '{orig}' → '{mapped}'")
else:
    print("\n⚠ Cannot create district mapping: One or both datasets are missing")


In [ ]:
## Step 3: Merge


# Merge the two dataframes on ['District_Name', 'Month'] using Inner Join
if jjm_df_clean is not None and health_df_clean is not None:
    print("Merging datasets...")
    print(f"  Water data shape: {jjm_df_clean.shape}")
    print(f"  Health data shape: {health_df_clean.shape}")
    
    # Check if required columns exist
    required_cols_jjm = ['District_Name', 'Month']
    required_cols_health = ['District_Name', 'Month']
    
    if all(col in jjm_df_clean.columns for col in required_cols_jjm) and \
       all(col in health_df_clean.columns for col in required_cols_health):
        
        # Perform inner join on District_Name and Month
        merged_df = pd.merge(
            jjm_df_clean,
            health_df_clean,
            on=['District_Name', 'Month'],
            how='inner',
            suffixes=('_water', '_health')
        )
        
        print(f"  ✓ Merge completed")
        print(f"  Merged data shape: {merged_df.shape}")
        print(f"  Columns: {list(merged_df.columns)}")
        
        # Show first few rows
        print(f"\n  First 5 rows:")
        display(merged_df.head(5))
        
    else:
        print(f"  ❌ Error: Required columns for merging not found")
        print(f"    Water data columns: {list(jjm_df_clean.columns)}")
        print(f"    Health data columns: {list(health_df_clean.columns)}")
        merged_df = None
else:
    print("❌ Cannot merge: One or both datasets are missing")
    merged_df = None


In [ ]:
## Step 4: Save


# Save the result to data/processed/final_panel_2019.csv
if merged_df is not None:
    output_file = Path(FILE_PATHS["data"]["processed"]) / "final_panel_2019.csv"
    output_file.parent.mkdir(parents=True, exist_ok=True)
    
    merged_df.to_csv(output_file, index=False, encoding='utf-8')
    
    print(f"✓ Saved merged data to: {output_file}")
    print(f"\nFinal DataFrame Shape: {merged_df.shape}")
    print(f"  Rows: {merged_df.shape[0]:,}")
    print(f"  Columns: {merged_df.shape[1]}")
    print(f"\nFirst 5 rows:")
    display(merged_df.head(5))
else:
    print("❌ Cannot save: Merged dataset is missing")


## Summary

The merged panel dataset has been created and saved to `data/processed/final_panel_2019.csv`.

**Dataset contains:**
- Synthetic water coverage data (FHTC_Coverage) merged with real health data (Cases)
- Filtered to include only "Inpatient" indicators
- District names standardized using fuzzy matching
- Merged on District_Name and Month using inner join

**Next Steps:**
- Use this dataset for econometric analysis
- Detect self-reporting bias in water coverage data
- Analyze the relationship between water coverage and health outcomes

